# Quiz 7

BEFORE YOU START THIS QUIZ:

1. Click on "Copy to Drive" to make a copy of the quiz,

2. Click on "Share",
    
3. Click on "Change" and select "Anyone with this link can edit"
    
4. Click "Copy link" and

5. Paste the link into [this Canvas assignment](https://canvas.olin.edu/courses/390/assignments/6389).

DO THIS BEFORE YOU START THE QUIZ.

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Here's a function that just might come in handy.

In [91]:
from statsmodels.nonparametric.smoothers_lowess import lowess

def make_lowess(series):
    """Use LOWESS to compute a smooth line.

    series: pd.Series

    returns: pd.Series
    """
    endog = series.values
    exog = series.index.values

    smooth = lowess(endog, exog)
    index, data = np.transpose(smooth)

    return pd.Series(data, index=index)

## Estimating a Proportion

Let's get some GSS data again.

In [92]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/' +
         'ElementsOfDataScience/raw/master/data/gss_eda.hdf')

In [93]:
import pandas as pd

gss = pd.read_hdf('gss_eda.hdf', 'gss')

In [94]:
gss.head()

I'm going to recode `GRASS` so 1 means yes (supporting legalization of cannabis) and 0 means no (opposed to legalization).

In [95]:
gss['grass'] = gss['GRASS'].replace(2, 0)

And I'll define `last` to be a Boolean Series that is `True` for the 2018 data, which is the last year in this dataset.

In [96]:
last = (gss['YEAR'] == 2018)
last.sum()

We can use `count` to see how many valid (non-NaN) responses we have from 2018.

In [97]:
n = gss[last]['grass'].count()
n

**Question 1:** Compute the fraction of valid responses that are `1` and assign that proportion to `p`. This is our estimate for the fraction of the population that supported legalization in 2018.

The following function from Notebook 13 simulates `n` coin tosses with probability `p`, and returns the number of heads.

In [99]:
def simulate_group_percent(n, p):
    xs = np.random.random(size=n)
    k = np.sum(xs < p)
    return k / n * 100

**Question 2:** Call `simulate_group_percent` 1000 times with the values of `n` and `p` we computed and save the results in a list.

You can use the following function to plot the sampling distribution of the estimate.

In [101]:
sns.kdeplot(t)

plt.xlabel('Percent supporting legalization');

**Question 3:** Compute the standard error and 80% confidence interval for the estimated percentage.

Note that I asked for an 80% interval, so that's different from what we've done previously.

## Interpolating a Date

Continuing the previous example, let's see how support for legalization has changed since 2000.
Here's a Boolean Series that's true for data from 2000 or later.

In [104]:
recent = gss['YEAR'] >= 2000
recent.sum()

And here's a Series that contains the fraction of people who said yes during each year.

In [105]:
series = gss[recent].groupby('YEAR')['grass'].mean()
smooth = make_lowess(series)

The following figure shows the results.

In [106]:
series.plot(style='o', color='C2', alpha=0.5)
smooth.plot(color='C2', alpha=0.5)

plt.ylabel('Fraction supporting legalization');

It's clear that support for legalization was less than 50% in 2000 and more than 50% in 2018, so we might wonder when it crossed the line.

We can use interpolation of the smoothed line to estimate when it happened.

In [107]:
from scipy.interpolate import interp1d

func = interp1d(smooth.values, smooth.index)
func

The result from `interp1d` is a function, which we can call like this.

In [108]:
func(0.5)

According to the interpolation, support for legalization reached 50% in late 2011.
The result is a NumPy array with a single value, but we can convert it to a float like this.

In [109]:
float(func(0.5))

At this point, we have a single estimate; now suppose we would also like a confidence interval to quantify the precision of that estimate.
We can do it by bootstrap resampling.
For example, here's a version of the function from Notebook 12 we can use to bootstrap the mean. 

In [110]:
def bootstrap_mean(df):
    bootstrapped = df.sample(n=len(df), replace=True)
    return bootstrapped['grass'].mean()

And we can call it like this:

In [111]:
bootstrap_mean(gss[recent])

**Question 4:** Write a function called `bootstrap_year` that takes a DataFrame and runs the following steps:

1. Use `sample` to generate a bootstrap sample of the rows.

2. Use `groupby` and `mean` to compute the fraction of support for legalization during each year.

3. Use `make_lowess` to fit a smooth curve to the data.

4. Use `interp1d` to interpolate the year support was 50%.

5. Return the result as a float (not an array).

Call your function like this to test it.

In [113]:
bootstrap_year(gss[recent])

**Question 5:** Call `bootstrap_year` 1000 times, save the results, and use them to compute a 90% CI for the estimated year.

This example shows that we can use bootstrap resampling to compute confidence intervals for pretty much any estimation process, even one like this that involves computations like local regression and interpolation.
This example would be all but impossible to do mathematically.

## Help a Redditor Out

A recent question on [r/stats](https://www.reddit.com/r/statistics/comments/tyi36l/q_what_is_the_correct_way_to_compare_these_two/) asks:

> [Q] What is the correct way to compare these two means?
> 
> Simple question but my statistics knowledge is rusty so I’m hoping to get some help. I am preparing a report and have water concentration data for two locations.
> 
> Location A: 214, 256, 218, 268. Mean 239.
> 
> Location B: 180, 181, 161. Mean 174.
> 
> I want to state “average concentration at location B is X% lower than at location A”. What is the correct formula to use to determine the percentage?
> 
> I do not need to speak to the statistical significance of the difference, but if I did, would I use a t-test?
> 
> Thanks!!

Here's the data in two arrays.

In [119]:
loc_a = np.array([214, 256, 218, 268])
loc_b = np.array([180, 181, 161])

**Question 6:** The question implies that the appropriate test statistic in this case is the *decrease* in the average at B expressed as a percentage of the average at A. Write a line of code to compute this test statistic for the given data.

Hint: Since the question is specifically about a decrease, you should not compute the absolute value of the difference.

**Question 7:** To see whether this apparent effect could be due to chance, let's use permutation. Write a function called `simulate_two_groups` that takes the two data arrays as parameters, combines the two groups, shuffles them, divides the shuffled values into two groups with the same sizes are the originals, then computes and returns the test statistic of the shuffled data.

Hint: Start with a similar function from Notebook 13.

Call your function like this to test it.

In [122]:
simulate_two_groups(loc_a, loc_b)

**Question 8:** Call your function 1000 times and save the results as a list. Use this list to estimate the p-value of the observed effect. Write a sentence to interpret the results; is it plausible that the observed effect it due to chance?

**Optional just for fun extra question:** If the original question asked about the *difference* between A and B, rather than the decrease at B relative to A, it would be more appropriate to use the absolute value of the difference as a test statistic.


*Elements of Data Science*

Copyright 2022 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)